<a href="https://colab.research.google.com/github/yonus07/Note-Agent/blob/main/data_cleaning_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
import os
from kagglehub import KaggleDatasetAdapter

# Download dataset
dataset_dir = kagglehub.dataset_download("ronnykym/online-store-sales-data")
print("Dataset directory:", dataset_dir)

# List files
files = os.listdir(dataset_dir)
print("Files:", files)

# Choose a file
file_path = files[0]  # or specify manually, e.g. "sales_data.csv"

# Load the file
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "ronnykym/online-store-sales-data",
    file_path,
)

df.head()


100%|██████████| 30.8k/30.8k [00:00<00:00, 30.7MB/s]

Extracting files...
Dataset directory: /root/.cache/kagglehub/datasets/ronnykym/online-store-sales-data/versions/1
Files: ['Sales-Export_2019-2020.csv']


Using Colab cache for faster access to the 'online-store-sales-data' dataset.


,country,order_value_EUR,cost,date,category,customer_name,sales_manager,sales_rep,device_type,order_id
0,Sweden,"17,524.02",14122.61,2/12/2020,Books,Goldner-Dibbert,Maxie Marrow,Madelon Bront,Mobile,70-0511466
1,Finland,"116,563.40",92807.78,9/26/2019,Games,Hilll-Vandervort,Hube Corey,Wat Bowkley,Mobile,28-6585323
2,Portugal,"296,465.56",257480.34,7/11/2019,Clothing,Larkin-Collier,Celine Tumasian,Smitty Culverhouse,PC,58-7703341
3,Portugal,"74,532.02",59752.32,4/2/2020,Beauty,Hessel-Stiedemann,Celine Tumasian,Aurelie Wren,PC,14-6700183
4,Spain,"178,763.42",146621.76,12/22/2019,Games,Johns and Sons,Emalia Dinse,Bertha Walbrook,Tablet,15-8765160


In [ ]:
import pandas as pd
import numpy as np

# Keep original safe
df_raw = df.copy()
df_clean = df.copy()

print("Shape:", df_clean.shape)
print("\nColumn names:")
print(df_clean.columns.tolist())

print("\nData types:")
print(df_clean.dtypes)

print("\nMissing values per column:")
print(df_clean.isna().sum())

Shape: (1000, 10)

Column names:
['country', ' order_value_EUR ', ' cost ', 'date', 'category', 'customer_name', 'sales_manager', 'sales_rep', 'device_type', 'order_id']

Data types:
country               object
 order_value_EUR      object
 cost                float64
date                  object
category              object
customer_name         object
sales_manager         object
sales_rep             object
device_type           object
order_id              object
dtype: object

Missing values per column:
country              0
 order_value_EUR     0
 cost                0
date                 0
category             0
customer_name        0
sales_manager        0
sales_rep            0
device_type          0
order_id             0
dtype: int64


In [ ]:
df_clean.columns = (
    df_clean.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("-", "_")
)
df_clean.columns


Index(['country', 'order_value_eur', 'cost', 'date', 'category',
       'customer_name', 'sales_manager', 'sales_rep', 'device_type',
       'order_id'],
      dtype='object')

In [ ]:
before_rows = df_clean.shape[0]
df_clean = df_clean.drop_duplicates()
after_rows = df_clean.shape[0]

print(f"Removed {before_rows - after_rows} duplicate rows")


Removed 0 duplicate rows


In [ ]:
# Select object (string) columns
str_cols = df_clean.select_dtypes(include="object").columns

# Strip spaces
df_clean[str_cols] = df_clean[str_cols].apply(lambda col: col.str.strip())

# Optionally, make some columns lowercase for consistency
# (Uncomment and change names if they exist in your data)
# for col in ['country', 'city', 'status', 'payment_method']:
#     if col in df_clean.columns:
#         df_clean[col] = df_clean[col].str.lower()

In [ ]:
# Find columns whose names contain 'date'
date_cols = [col for col in df_clean.columns if "date" in col.lower()]
print("Detected date-like columns:", date_cols)

for col in date_cols:
    df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")

Detected date-like columns: ['date']


In [ ]:
# Try to convert any object columns that look numeric
for col in df_clean.columns:
    if df_clean[col].dtype == "object":
        # if many values look like numbers, convert
        try:
            df_clean[col] = pd.to_numeric(df_clean[col].str.replace(",", ""), errors="ignore")
        except AttributeError:
            # column is not string-like, skip
            pass

# Check numeric columns
num_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
print("Numeric columns:", num_cols.tolist())


Numeric columns: ['order_value_eur', 'cost']


/tmp/ipython-input-858661495.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_clean[col] = pd.to_numeric(df_clean[col].str.replace(",", ""), errors="ignore")


In [ ]:
df_clean.isna().sum().sort_values(ascending=False)

,0
country,0
order_value_eur,0
cost,0
date,0
category,0
customer_name,0
sales_manager,0
sales_rep,0
device_type,0
order_id,0


In [ ]:
# 6.1 Drop rows where key columns are missing (change names to yours)
key_cols = []
for candidate in ['order_id', 'invoice_no', 'order_date']:
    if candidate in df_clean.columns:
        key_cols.append(candidate)

if key_cols:
    df_clean = df_clean.dropna(subset=key_cols)

# 6.2 Fill numeric columns with median
num_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
for col in num_cols:
    median_val = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(median_val)

# 6.3 Fill categorical columns with mode (most frequent value)
cat_cols = df_clean.select_dtypes(include="object").columns
for col in cat_cols:
    if df_clean[col].isna().sum() > 0:
        mode_val = df_clean[col].mode()
        if not mode_val.empty:
            df_clean[col] = df_clean[col].fillna(mode_val[0])
        else:
            df_clean[col] = df_clean[col].fillna("unknown")


In [ ]:
print("Shape after cleaning:", df_clean.shape)
print("\nNulls after cleaning:")
print(df_clean.isna().sum())

df_clean.head()
df_clean.info()

Shape after cleaning: (1000, 10)

Nulls after cleaning:
country            0
order_value_eur    0
cost               0
date               0
category           0
customer_name      0
sales_manager      0
sales_rep          0
device_type        0
order_id           0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   country          1000 non-null   object        
 1   order_value_eur  1000 non-null   float64       
 2   cost             1000 non-null   float64       
 3   date             1000 non-null   datetime64[ns]
 4   category         1000 non-null   object        
 5   customer_name    1000 non-null   object        
 6   sales_manager    1000 non-null   object        
 7   sales_rep        1000 non-null   object        
 8   device_type      1000 non-null   object        
 9   order_id         1000 non-n

In [ ]:
df_clean.to_csv("online_store_sales_cleaned.csv", index=False)